In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from util.load import DataLoader as Loader
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import pandas as pd
from sklearn.model_selection import train_test_split


torch.manual_seed(42)
np.random.seed(42)

In [23]:
loader = Loader(data_dir="../data")

sequences, targets = loader.load_daily_sequence_data("kxhighny", verbose=True)
train_sequences, val_sequences, train_targets, val_targets = train_test_split(sequences, targets, test_size=0.2, random_state=42)

Loading kxhighny for 2025-03-25: 100%|██████████| 38/38 [00:09<00:00,  3.94it/s]


In [24]:
print(train_sequences.shape, val_sequences.shape)
print(train_targets.shape, val_targets.shape)


(33969, 100, 61) (8493, 100, 61)
(33969, 6) (8493, 6)


In [26]:

class VariableLengthLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(VariableLengthLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
        )
        self.fc = nn.Linear(hidden_size, output_size)
        self.leaky_relu = nn.LeakyReLU(0.1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        packed_output, (hidden, _) = self.lstm(x, (h0, c0))

        out = hidden[-1, :, :]
        out = self.fc(out)
        out = self.leaky_relu(out)
        out = self.softmax(out)
        return out

In [27]:
class SequenceDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences  # List of variable-length sequences
        self.targets = targets  # List of targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]


In [30]:
train_dataset = SequenceDataset(train_sequences, train_targets)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

val_dataset = SequenceDataset(val_sequences, val_targets)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True)



In [29]:
input_size = 61  # Number of features
hidden_size = 128
num_layers = 2
output_size = 1
learning_rate = 0.001
num_epochs = 50

# Initialize model, loss function, and optimizer
model = VariableLengthLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Step 6: Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for sequences, targets in train_loader:
        # Zero the gradients
        # make the sequences a float32 tensor
        sequences = sequences.float()
        targets = targets.float()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(sequences)

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for sequences, lengths, targets in val_loader:
            outputs = model(sequences, lengths)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    # Print statistics
    if (epoch + 1) % 5 == 0:
        print(
            f"Epoch [{epoch+1}/{num_epochs}], "
            f"Train Loss: {train_loss/len(train_loader):.4f}, "
            f"Val Loss: {val_loss/len(val_loader):.4f}"
        )

/opt/anaconda3/envs/jump/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([256, 6])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 